# RTF Model for predicting Racket with Multi Features - Vibrations

In [6]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from scipy.fft import fft

def extract_features(signal):
    # Extraction de caractéristiques statistiques et fréquentielles
    mean = np.mean(signal)
    std = np.std(signal)
    max_val = np.max(signal)
    min_val = np.min(signal)
    energy = np.sum(signal**2)
    
    # Fréquences : on prend les 10 premières composantes FFT (hors composante continue)
    fft_vals = np.abs(fft(signal))[1:11]
    
    return np.hstack([mean, std, max_val, min_val, energy, fft_vals])

# Chemin vers ton dossier
folder_path = "all_vib"

X = []
y = []

# Chargement et traitement des fichiers
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        data = pd.read_csv(file_path)
        if data.shape[0] == 2400:
            signal = data.values.flatten()
            features = extract_features(signal)
            X.append(features)
            label = filename.split("_")[1]
            y.append(label)

X = np.array(X)
y = np.array(y)

# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Recherche des meilleurs hyperparamètres
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# Affichage du rapport de classification
print("Meilleurs hyperparamètres :", grid.best_params_)
print("Rapport de classification :")
print(classification_report(y_test, y_pred))


Meilleurs hyperparamètres : {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Rapport de classification :
              precision    recall  f1-score   support

          RB       0.69      0.71      0.70        35
          RO       0.80      0.65      0.72        49
          RR       0.63      0.75      0.69        32
          RV       0.78      0.82      0.80        39

    accuracy                           0.73       155
   macro avg       0.73      0.73      0.73       155
weighted avg       0.74      0.73      0.73       155



In [7]:
from sklearn.metrics import accuracy_score

# Affichage de la précision
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {accuracy:.2%}")


Précision du modèle : 72.90%
